In [1]:
import pandas as pd
import numpy as np

Survey lessons learned:
1. We should require people to mark I have read this post
   - There is a typo between "I have read this post" and "I have read this post."
2. Make sure the cases of the Strongly Agree to Strongly Disagree are all the same

What I learned:
1. Q8B should not be reverse encoded

## Load Data

In [2]:
jewAB = pd.read_csv("./data/jewAB.csv")

In [3]:
jewBA = pd.read_csv("./data/jewBA.csv")

In [4]:
muslimAB = pd.read_csv("./data/muslimAB.csv")

In [5]:
muslimBA = pd.read_csv("./data/muslimBA.csv")

In [6]:
# Show that the questions are the same just a different order between the two datasets
assert jewAB['Q1A'][0] == jewBA['Q1A'][0]

In [7]:
print(jewAB.columns)

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'QID15', 'Q16', 'Q17', 'Q18',
       'Q19', 'Q1A', 'Q2A', 'Q3A', 'Q4A', 'Q5A', 'Q6A', 'Q7A', 'Q8A', 'N0',
       'N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'Narrative Reflection', 'Q1B',
       'Q2B', 'Q3B', 'Q4B', 'Q5B', 'Q6B', 'Q7B', 'Q8B'],
      dtype='object')


In [8]:
narrative_columns = ['N0','N1','N2','N3','N4','N5','N6','N7']
def find_column_with_value(row):
    for col in ['N0','N1','N2','N3','N4','N5','N6','N7']:
        if row[col] == "I have read this post" or row[col] == "I have read this post.":
            return col
    return 'Bad Data'



In [9]:
def analyze_df(df):
    print(len(df))
    for c in narrative_columns:
        vc = df[c].value_counts().get("I have read this post.")
        v2 = df[c].value_counts().get("I have read this post")
        value = vc if v2 == None else v2
        print(f"col: {c} count: {value}")

In [10]:
analyze_df(jewAB)

33
col: N0 count: 6
col: N1 count: 4
col: N2 count: 6
col: N3 count: 2
col: N4 count: 1
col: N5 count: 3
col: N6 count: 2
col: N7 count: 6


In [11]:
jewAB['Narrative'] = jewAB.apply(find_column_with_value, axis=1)
jewBA['Narrative'] = jewBA.apply(find_column_with_value, axis=1)
muslimAB['Narrative'] = muslimAB.apply(find_column_with_value, axis=1)
muslimBA['Narrative'] = muslimBA.apply(find_column_with_value, axis=1)

### Data Processing Steps:
1. Encode what Narrative is assigned (filter out those that did not mark they read the post)
2. Encode the Likert Scales
3. Apply Reverse encoding

In [12]:
def convert_to_num_col(col_list):
    nlist = []
    for col in col_list:
        nlist.append(str(col+"n"))
    return nlist

In [13]:
def create_likert_values(df,lc,ld):
    for col in lc:
        name = col + "n"
        df[name] = df[col].apply(lambda x: None if not (x in ld) else ld[x])
    return df

In [14]:
def calculate_ats(df,sA,sB,gA,gB):
    # Calculate Attitudes
    df["sectA"] = df[convert_to_num_col(sA)].mean(axis=1)
    df["sectB"] = df[convert_to_num_col(sB)].mean(axis=1)
    df["genA"] = df[convert_to_num_col(gA)].mean(axis=1)
    df["genB"] = df[convert_to_num_col(gB)].mean(axis=1)
    
    return df

In [15]:
likert_dict = {"Strongly Disagree":1,"Strongly agree":5,"Somewhat disagree":2,"Neither agree nor disagree":3,"Somewhat agree":4}
likert_cols = ['Q1B', 'Q2B', 'Q3B', 'Q4B', 'Q5B', 'Q6B', 'Q7B','Q8B','Q1A', 'Q2A', 'Q3A', 'Q4A', 'Q5A', 'Q6A', 'Q7A','Q8A']
likert_colsn = ['Q1Bn', 'Q2Bn', 'Q3Bn', 'Q4Bn', 'Q5Bn', 'Q6Bn', 'Q7Bn','Q8Bn','Q1An', 'Q2An', 'Q3An', 'Q4An', 'Q5An', 'Q6An', 'Q7An','Q8An']

In [16]:
# Revese Encode
reverse_columns = ['Q3A','Q4A','Q7A','Q1B','Q3B','Q7B']
reverse_likert_dict = {"Strongly Disagree":5,"Strongly agree":1,"Somewhat disagree":4,"Neither agree nor disagree":3,"Somewhat agree":2}


In [17]:
print(jewAB["Q8B"][0])

The impact on civilians in this conflict is excessively severe.


In [18]:
filtered_jewAB = create_likert_values(jewAB,likert_cols,likert_dict)
filtered_jewAB = create_likert_values(jewAB,reverse_columns,reverse_likert_dict)

filtered_jewBA = create_likert_values(jewBA,likert_cols,likert_dict)
filtered_jewBA = create_likert_values(jewBA,reverse_columns,reverse_likert_dict)

filtered_muslimAB = create_likert_values(muslimAB,likert_cols,likert_dict)
filtered_muslimAB = create_likert_values(muslimAB,reverse_columns,reverse_likert_dict)

filtered_muslimBA = create_likert_values(muslimBA,likert_cols,likert_dict)
filtered_muslimBA = create_likert_values(muslimBA,reverse_columns,reverse_likert_dict)




In [19]:
print(jewAB)

                                            StartDate  \
0                                          Start Date   
1   {"ImportId":"startDate","timeZone":"Pacific/Ho...   
2                                 2024-04-27 13:41:35   
3                                 2024-04-27 13:37:30   
4                                 2024-04-28 07:45:31   
5                                 2024-04-28 08:04:02   
6                                 2024-04-28 11:08:31   
7                                 2024-04-28 11:46:57   
8                                 2024-04-28 11:58:31   
9                                 2024-04-28 11:49:16   
10                                2024-04-28 12:02:15   
11                                2024-04-28 12:00:51   
12                                2024-04-28 12:19:58   
13                                2024-04-28 12:26:43   
14                                2024-04-28 11:50:26   
15                                2024-04-28 13:13:50   
16                             

## Data Analysis Steps
1. Calculate chronbach's alpha
2. Filter through each of the narratives to see if there is a significant change in sectarian attitudes before and after being presented an narrative

In [20]:
import pingouin as pg

In [21]:
sectA = ['Q1A','Q2A','Q3A','Q4A'] # Remove the average person question (Q3A) 
genA = ['Q5A','Q6A','Q7A','Q8A']
sectB = ['Q1B','Q2B','Q3B','Q4B']
genB = ['Q5B','Q6B','Q7B','Q8B']

In [22]:
filtered_jewAB = calculate_ats(filtered_jewAB,sectA,sectB,genA,genB)
filtered_jewBA = calculate_ats(filtered_jewBA,sectA,sectB,genA,genB)
filtered_muslimAB = calculate_ats(filtered_muslimAB,sectA,sectB,genA,genB)
filtered_muslimBA = calculate_ats(filtered_muslimBA,sectA,sectB,genA,genB)

name_list = ["Jew A -> B", "Jew B -> A", "Muslim A -> B", "Muslim B -> A"]
data_list = [filtered_jewAB,filtered_jewBA,filtered_muslimBA,filtered_muslimAB]
before_after_sect_list = [("sectA","sectB"),("sectB","sectA"),("sectA","sectB"),("sectB","sectA")]
before_after_gen_list = [("genA","genB"),("genB","genA"),("genA","genB"),("genB","genA")]

In [23]:
# Combinations to check:
# Dataframes: jAB, jBA, mAB, mBA
# Groups: first, last, both, first + gen, last + gen, all, gen + first, gen + last, gen + both

sect_A = convert_to_num_col(sectA)
sect_B = convert_to_num_col(sectB)
sect_all = sect_A + sect_B

gen_A = convert_to_num_col(genA)
gen_B = convert_to_num_col(genB)
gen_all = gen_A + gen_B

sect_gen_A = sect_A + gen_A
sect_gen_B = sect_B + gen_B
sect_gen_all = sect_all + gen_all

def calculate_cbalpha(df,columns,name_of_analysis,show_output=False):
    subdf = df.loc[:,columns]
    subdf_cleaned = subdf.dropna()
    if show_output:
        print(subdf_cleaned)
    alpha = pg.cronbach_alpha(data=subdf_cleaned)
    print(f"{name_of_analysis}: {alpha}")
    return alpha

In [24]:
for i in range(0,len(name_list)):
    name = name_list[i]
    data = data_list[i]
    print(f"-------- {name} --------")
    calculate_cbalpha(data,sect_A,str(f"{name} sect_A"),show_output=False)
    calculate_cbalpha(data,sect_B,str(f"{name} sect_B"))
    calculate_cbalpha(data,sect_all,str(f"{name} sect_all"))
    calculate_cbalpha(data,gen_A,str(f"{name} gen_A"))
    calculate_cbalpha(data,gen_B,str(f"{name} gen_B"),show_output=False)
    calculate_cbalpha(data,gen_all,str(f"{name} gen_all"))
    calculate_cbalpha(data,sect_gen_A,str(f"{name} sect_gen_A"))
    calculate_cbalpha(data,sect_gen_B,str(f"{name} sect_gen_B"))
    calculate_cbalpha(data,sect_gen_all,str(f"{name} sect_gen_all"))

-------- Jew A -> B --------
Jew A -> B sect_A: (0.9302325581395354, array([0.866, 0.966]))
Jew A -> B sect_B: (0.5633463737586517, array([0.244, 0.769]))
Jew A -> B sect_all: (0.8560521292700587, array([0.752, 0.927]))
Jew A -> B gen_A: (0.4763877822045153, array([-0.035,  0.77 ]))
Jew A -> B gen_B: (0.37499999999999983, array([-0.082,  0.67 ]))
Jew A -> B gen_all: (0.2922936935675789, array([-0.297,  0.678]))
Jew A -> B sect_gen_A: (0.6321152494729445, array([0.284, 0.847]))
Jew A -> B sect_gen_B: (0.5668677187474179, array([0.291, 0.764]))
Jew A -> B sect_gen_all: (0.7761487121412909, array([0.583, 0.905]))
-------- Jew B -> A --------
Jew B -> A sect_A: (0.6313559322033899, array([0.283, 0.826]))
Jew B -> A sect_B: (0.4790270538595185, array([0.106, 0.721]))
Jew B -> A sect_all: (0.7234111543450066, array([0.519, 0.862]))
Jew B -> A gen_A: (0.6577868852459017, array([0.335, 0.846]))
Jew B -> A gen_B: (0.23134328358208936, array([-0.32 ,  0.589]))
Jew B -> A gen_all: (0.688952017745

### Look at Effect of Narrative on Attitudes

In [25]:
from scipy.stats import ttest_rel

In [26]:
# name_list = ["Jew A -> B", "Jew B -> A", "Muslim A -> B", "Muslim B -> A"]
# data_list = [filtered_jewAB,filtered_jewBA,filtered_muslimBA,filtered_muslimAB]
# before_after_sect_list = [(sectA,sectB),(sectB,sectA),(sectA,sectB),(sectB,sectA)]
# before_after_gen_list = [(genA,genB),(genB,genA),(genA,genB),(genB,genA)]

In [29]:
def print_result(name,narrative,size,p_value_label, p_value):
    if p_value < 0.05:
        print(f"{name} Narrative: {narrative} {p_value_label} # of Data Points: {size} p-value: {p_value} Result: Significant")
    else:
        print(f"{name} Narrative: {narrative} {p_value_label} # of Data Points: {size} p-value: {p_value} Result: -")
        

In [30]:
# Combinations to check
# Dataframes: jAB, jBA, mAB, mBA
# Narratives: 'N0','N1','N2','N3','N4','N5','N6','N7' (narrative_columns)
# Groups: before_after_sect, before_after_gen

for i in range(0,len(name_list)):
    name = name_list[i]
    data = data_list[i]
    sect = before_after_sect_list[i]
    gen = before_after_gen_list[i]
    
    for narrative in narrative_columns:
        subdf = data[data["Narrative"] == narrative]
        size = len(subdf)
        sect_t_statistic, sect_p_value = ttest_rel(subdf[sect[0]], subdf[sect[1]])
        print_result(name,narrative,size,"Sect.", sect_p_value)
        
        gen_t_statistic, gen_p_value = ttest_rel(subdf[gen[0]], subdf[gen[1]])
        print_result(name,narrative,size,"Genl.", gen_p_value)
        

Jew A -> B Narrative: N0 Sect. # of Data Points: 6 p-value: 0.017962884609943948 Result: Significant
Jew A -> B Narrative: N0 Genl. # of Data Points: 6 p-value: 0.19328050284729878 Result: -
Jew A -> B Narrative: N1 Sect. # of Data Points: 4 p-value: 0.39100221895577053 Result: -
Jew A -> B Narrative: N1 Genl. # of Data Points: 4 p-value: 0.7176856442107861 Result: -
Jew A -> B Narrative: N2 Sect. # of Data Points: 6 p-value: 0.07056563534883982 Result: -
Jew A -> B Narrative: N2 Genl. # of Data Points: 6 p-value: 0.5270761892087815 Result: -
Jew A -> B Narrative: N3 Sect. # of Data Points: 2 p-value: 0.2951672353008664 Result: -
Jew A -> B Narrative: N3 Genl. # of Data Points: 2 p-value: 0.20483276469913345 Result: -
Jew A -> B Narrative: N4 Sect. # of Data Points: 1 p-value: nan Result: -
Jew A -> B Narrative: N4 Genl. # of Data Points: 1 p-value: nan Result: -
Jew A -> B Narrative: N5 Sect. # of Data Points: 3 p-value: 0.09273529127344506 Result: -
Jew A -> B Narrative: N5 Genl. # o

## Analysis

Since none of the Significant results were repeated when the order of the questions presented was changed. For example, Narrative 4 had a statistically significant change in attitudes in the survey where the set of B questions were presented before the set of A questions. That statistically significant result was not replicated when the set of A questions were presented first (the p-value was in fact 0.68).